In [4]:
import numpy as np
# import matplotlib.pyplot as plt
import os
import sys
import re

In [3]:
a = np.load('../error_growth/assemble/linear_beta0.5nu4e-05t400initial400dt0.01repeat10000perturbed_dim13_26910.npy')
# plt.plot(abs(a[0, :]), abs(a[1, :]))
a.shape

(2, 40001)

## error_growthのnpyの結合

In [2]:
# "../error_growth/assemble/" から.npyファイルを全て読み込む
directory = "../error_growth/assemble/"
files = os.listdir(directory)
files_file = [f for f in files if os.path.isfile(os.path.join(directory, f))]
files_npy = [f for f in files_file if f[-4:] == '.npy']

# ファイル名からパラメータを取得
beta, nu, t, t_initial, dt = re.search(r'beta(\d+\.\d+)nu(\d+e-\d+)t(\d+)initial(\d+)dt(\d+\.\d+)', files_npy[0]).groups()
perturbed_dim = re.search(r'perturbed_dim(\d+)', files_npy[0]).group(1)
repetition = 0
time = np.load(directory + files_npy[0])[1,]
average = np.zeros_like(time)

# ファイルを読み込んで平均を取る
for i in range(len(files_npy)):
    beta_local, nu_local, t_local, t_initial_local, dt_local = re.search(r'beta(\d+\.\d+)nu(\d+e-\d+)t(\d+)initial(\d+)dt(\d+\.\d+)', files_npy[i]).groups()
    perturbed_dim_local = re.search(r'perturbed_dim(\d+)', files_npy[i]).group(1)
    # パラメータが一致しているか確認
    if beta_local != beta or nu_local != nu or t_local != t or t_initial_local != t_initial or dt_local != dt or perturbed_dim_local != perturbed_dim:
        print("Parameter is not matched")
        continue
    repetition_local = int(re.search(r'repeat(\d+)', files_npy[i]).group(1))
    repetition += repetition_local
    average += np.load(directory + files_npy[i])[0, ] * repetition_local
    
average /= repetition
error_growth = (np.log(average[2:, ]) - np.log(average[:-2, ])) / (2*float(dt))

arr = np.array([average[1:-1], error_growth, time[1:-1]])
# 保存
np.save("../error_growth/linear_beta" + beta + "nu" + nu + "t" + t + "initial" + t_initial + "dt" + dt + "repeat" + str(repetition) + "perturbed_dim" + perturbed_dim + ".npy", arr)
    